In [7]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-09-07 11:25:46--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2024-09-07 11:25:46 (11.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [10]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [13]:
print(f"length of dataset in characters: {len(text)}")

length of dataset in characters: 1115394


In [15]:
# first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [24]:
# extracting vocabulary
characters = sorted(list(set(text)))
vocab_size = len(characters)
print(f'vocab size: {vocab_size}')
print(''.join(characters))

vocab size: 65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [26]:
# build text tokenizer
stoi = { c:idx for idx, c in enumerate(characters) }
itos = { idx:c for idx, c in enumerate(characters) }
# encoder: take a string and output a list of integers
encode = lambda s: [stoi[c] for c in s] 
# decoder: take a list of integers and output a string
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('as you like it!'))
print(decode(encode('as you like it!')))

[39, 57, 1, 63, 53, 59, 1, 50, 47, 49, 43, 1, 47, 58, 2]
as you like it!


In [29]:
# encode entire dataset and store as torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [30]:
# split the dataset
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [32]:
# visualizing input
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [36]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} the target is {target}')

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [46]:
# generating a batch of data

torch.manual_seed(1337)
batch_size = 4 # number of independent sequences we process in parallel
block_size = 8 # maximum context length for predictions

def get_batch(split):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(train_data)-block_size, (batch_size, ))
    x = torch.stack([data[x:x+block_size] for x in ix])
    y = torch.stack([data[x+1:x+block_size+1] for x in ix])
    return x, y

xb, yb = get_batch('train')
print(f'inputs {xb.shape} \n {xb}')
print(f'labels {yb.shape} \n {yb}')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when input is {context} the target is {target}')
        

inputs torch.Size([4, 8]) 
 tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
labels torch.Size([4, 8]) 
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when input is tensor([24]) the target is 43
when input is tensor([24, 43]) the target is 58
when input is tensor([24, 43, 58]) the target is 5
when input is tensor([24, 43, 58,  5]) the target is 57
when input is tensor([24, 43, 58,  5, 57]) the target is 1
when input is tensor([24, 43, 58,  5, 57,  1]) the target is 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]) the target is 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target is 39
when input is tensor([44]) the target is 53
when input is tensor([44, 53]) the target is 56
when input is tensor([44, 53, 56]) the target is 1
w

In [79]:
# Establishing baseline: Bigram model
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

class Bigram(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(num_embeddings=vocab_size, embedding_dim=vocab_size)

    def forward(self, idx, targets=None):
        """
        idx [B,T], targets dimension [B,T]
        logits dimentsion [B,T,C] where, B is batch, T is Time, C is the embedding dimension
        CE expects:
            Input (logits) to be of shape [N, C], where N is the number of samples and C is the number of classes.
            The target should be of shape [N], where each value is the target class for the corresponding sample.
        O/P logits [B*T, C] OR [B,T,C], loss
        """
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits, targets = logits.view(B*T, C), targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        """
        I/P idx dim [B,T], O/P idx dim [B, T+max_new_tokens]
        new tokens are added in the Time dimension for each example in the Batch
        """
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # plucking out the logits of the last element in T having dim (B, C)
            probs = F.softmax(logits, dim=-1) # dim (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # dim (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # dim (B, T+1)
        return idx

model = Bigram(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

print(decode(model.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [70]:
# expected loss
import math
-math.log(1/65) # natural log

4.174387269895637

Means the initial predictions are not uniform. They've some entropy.

In [81]:
# training Bigram
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [90]:
for k in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    # evaluate the loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(loss)

tensor(2.2748, grad_fn=<NllLossBackward0>)


In [92]:
print(decode(model.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=300)[0].tolist()))


t as yo;

EONTheset wef ars lid, wfe h'GSern pof?O, Onende n mshabim banas; Andalt nea f hil wno rth al.
w Sind alawis f
s, outrd all pat susu ueidog ncapr,
Whe d tim d w; buce IZme wnlo t or hes wo money w!Pia, Whe f te p. wilof Arerteave'lat arafou:
WSe f VOLUFINCa:

Ylare t ve bebeobunovil
Twene 


In [137]:
# Self-attention
# approach 1: averaging previous tokens in incremental fashion

# method 1
torch.manual_seed(1338)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [138]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xbow[b,t] = x[b,:t+1].mean(dim=0)

In [112]:
print(xbow[0])

tensor([[-1.3113, -1.0017],
        [-1.2728, -0.4360],
        [-1.0202, -0.7129],
        [-0.9331, -0.4884],
        [-0.5591, -0.3279],
        [-0.6985, -0.2544],
        [-0.5469,  0.1418],
        [-0.5745,  0.0050]])


In [113]:
print(x[0])

tensor([[-1.3113, -1.0017],
        [-1.2342,  0.1297],
        [-0.5150, -1.2666],
        [-0.6719,  0.1851],
        [ 0.9367,  0.3139],
        [-1.3950,  0.1132],
        [ 0.3622,  2.5192],
        [-0.7672, -0.9529]])


In [131]:
# method 2

a = torch.tril(torch.ones((3,3), dtype=torch.float32))
a = a/torch.sum(a, dim=1, keepdim=True)
b = torch.randn((3,2), dtype=torch.float32)
c = a@b

In [139]:
wei = torch.tril(torch.ones((T, T)))
wei = wei/torch.sum(wei, dim=1, keepdim=True)
xbow2 = wei @ x # (T,T) @ (B,T,C) --> (B, T, C)

In [140]:
xbow[0]

tensor([[-1.3113, -1.0017],
        [-1.2728, -0.4360],
        [-1.0202, -0.7129],
        [-0.9331, -0.4884],
        [-0.5591, -0.3279],
        [-0.6985, -0.2544],
        [-0.5469,  0.1418],
        [-0.5745,  0.0050]])

In [141]:
xbow2[0]

tensor([[-1.3113, -1.0017],
        [-1.2728, -0.4360],
        [-1.0202, -0.7129],
        [-0.9331, -0.4884],
        [-0.5591, -0.3279],
        [-0.6985, -0.2544],
        [-0.5469,  0.1418],
        [-0.5745,  0.0050]])

In [164]:
# method 3
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
xbow3[0]

tensor([[-1.3113, -1.0017, -1.2342,  0.1297, -0.5150, -1.2666, -0.6719,  0.1851,
          0.9367,  0.3139, -1.3950,  0.1132,  0.3622,  2.5192, -0.7672, -0.9529,
          1.1362, -1.0822, -1.1951, -0.2026,  1.4871, -0.3252, -1.2037,  0.4464,
         -0.9426, -0.6644,  0.4842, -0.5694,  0.5191, -1.9669,  0.8905,  0.8794],
        [-0.6606, -0.4705, -0.9986,  0.9048, -0.6031, -1.2426,  0.1387, -0.6026,
          0.7580,  0.3283, -1.3313,  0.6293, -0.2586,  1.4394, -1.1606, -0.6422,
          0.4951, -0.5413, -1.1707,  0.0407,  1.5133, -0.7808, -1.3433,  0.2280,
          0.1571, -0.3397,  0.1989, -1.0604,  0.4132, -1.9087,  0.1750,  0.1689],
        [-0.7987, -0.6624, -0.8659,  0.0878, -0.2328, -1.2967, -0.1081, -0.4746,
          0.5488, -0.1270, -1.1340,  0.0086, -0.0391,  0.3386, -0.7291, -0.1274,
          0.3893, -0.5096, -1.1740,  0.2090,  0.8383, -0.6148, -0.8194, -0.2207,
          0.0591, -0.2335,  0.0909, -0.5183,  0.5834, -1.2259, -0.3425,  0.5464],
        [-0.6520, -0.4619

In [175]:
# method 3
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # B,T,head_size
q = query(x) # B,T,head_size
wei = q @ k.transpose(-2, -1) # (B,T,head_size @ B,head_size,T) --> (B,T,T)

tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

In [176]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [5]:
7//4

1

In [7]:
import torch.nn as nn

class LayerNorm1D(nn.Module):
    def __init__(self, dim, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

    def __call__(self, x):
        xmean = x.mean(1, keepdim=True) # batch mean
        xvar = x.var(1, keepdim=True) # batch variance
        xhat = (x - xmean)/torch.sqrt(xvar + self.eps)
        self.out = self.gamma*xhat + self.beta
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]

In [11]:
[[1,2,3]]


[[1, 2, 3]]